In [20]:
import math
import pandas as pd

def construir_matriz(archivo):
    matriz_termino_documento = {}
    documentos = []

    try:
        with open(archivo, 'r', encoding='utf-8') as file:
            index = 1
            for linea in file:
                if not linea.strip():
                    continue

                documentos.append(index)
                palabras = limpiar(linea)

                for palabra in palabras:
                    if palabra not in matriz_termino_documento:
                        matriz_termino_documento[palabra] = {}

                    if index not in matriz_termino_documento[palabra]:
                        matriz_termino_documento[palabra][index] = 1
                    else:
                        matriz_termino_documento[palabra][index] += 1

                index += 1

        for palabra in matriz_termino_documento:
            for doc_id in documentos:
                if doc_id not in matriz_termino_documento[palabra]:
                    matriz_termino_documento[palabra][doc_id] = 0

    except FileNotFoundError:
        print(f"Error: El archivo '{archivo}' no se encontró.")

    return matriz_termino_documento, documentos

def palabras_en_documento(matriz_termino_documento):
    total_palabras_en_documento = {}
    for palabra in matriz_termino_documento:
        for doc, frecuencia in matriz_termino_documento[palabra].items():
            if doc not in total_palabras_en_documento:
                total_palabras_en_documento[doc] = 0
            total_palabras_en_documento[doc] += frecuencia
    return total_palabras_en_documento

def calcular_tf_idf(matriz, total_palabras_en_documento, total_documentos):
    tf_total = {}   # Frecuencia relativa de cada palabra en cada documento
    idf_total = {}  # Inversa de la frecuencia de documentos para cada palabra

    for palabra in matriz:
        tf_total[palabra] = {}

        documentos_con_palabra = 0
        for doc in matriz[palabra]:
            if matriz[palabra][doc] > 0:
                documentos_con_palabra += 1

        if documentos_con_palabra == 0:
            idf = 0
        else:
            idf = math.log(total_documentos / documentos_con_palabra)

        idf_total[palabra] = idf

        for doc in matriz[palabra]:
            total_palabras = total_palabras_en_documento.get(doc, 0)
            if total_palabras > 0:
                tf = matriz[palabra][doc] / total_palabras
            else:
                tf = 0
            tf_total[palabra][doc] = tf

    return tf_total, idf_total


def limpiar(texto):
    texto = texto.lower()
    caracteres_validos = "abcdefghijklmnopqrstuvwxyz0123456789áéíóúüñ"
    texto_limpio = ""
    
    for caracter in texto:
        if caracter in caracteres_validos:
            texto_limpio += caracter
        else:
            texto_limpio += " "
    
    palabras = texto_limpio.split()
    return palabras


matriz_termino_documento, documentos = construir_matriz('/kaggle/input/corpus3/01_corpus_turismo.txt')
total_palabra_en_documento = palabras_en_documento(matriz_termino_documento)
total_documentos = len(documentos)  

tf_total, idf_total = calcular_tf_idf(matriz_termino_documento, total_palabra_en_documento, total_documentos)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Matriz término-documento 
df_matriz = pd.DataFrame(matriz_termino_documento).T.fillna(0).astype(int)
print("\n📄 MATRIZ TÉRMINO-DOCUMENTO:")
print(df_matriz)

# TF 
df_tf = pd.DataFrame(tf_total).T
print("\n📊 MATRIZ TF:")
print(df_tf.round(4))

# IDF 
df_idf = pd.DataFrame.from_dict(idf_total, orient='index', columns=['IDF'])
print("\n📉 VALORES IDF:")
print(df_idf.round(4))







📄 MATRIZ TÉRMINO-DOCUMENTO:
                1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16
ecuador          1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
es               1   0   0   0   0   1   0   0   1   0   0   0   0   0   0   0
un               1   0   1   0   0   0   0   0   1   0   0   0   0   0   0   0
país             1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
megadiverso      1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
con              1   0   1   0   0   0   1   0   0   1   1   0   0   0   0   0
playas           1   0   0   0   1   0   0   0   0   1   0   0   0   0   0   0
montañas         1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
y                1   0   0   1   1   1   1   0   1   1   1   1   1   1   1   1
selvas           1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
ideales          1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
para             1   0 